In [129]:
using Random
using Distributions

In [205]:
abstract type AbstractBlk end

struct Minor
    label::String
end

struct Genesis <: AbstractBlk end

struct Blk <: AbstractBlk
    index::Int
    minor::Minor
    prev::AbstractBlk
end

function Base.show(io::IO, b::Genesis)
    Base.show(io, 0)
end

function Base.show(io::IO, b::Blk)
    Base.show(io, (b.index, b.minor.label))
end

struct Queue
    sender::Minor
    receiver::Minor
    elem::Vector{AbstractBlk}
    
    Queue(sender, receiver) = new(sender, receiver, Vector{AbstractBlk}())
end

In [206]:
getid(b::Blk) = b.index
getid(b::Genesis) = 0

getid (generic function with 2 methods)

In [207]:
getreceiver(q::Queue) = q.receiver

function receiveblk!(q::Queue, i::Int)::AbstractBlk
    b = q.elem[i]
    deleteat!(q.elem, i)
    b
end

getlength(q::Queue) = length(q.elem)

getlength (generic function with 1 method)

In [208]:
function sendblk!(q::Queue, b::AbstractBlk)
    push!(q.elem, b)
end

sendblk! (generic function with 1 method)

In [209]:
function event_genBlk!(x::Minor, workblk::Dict{Minor,AbstractBlk}, queue::Vector{Queue}, blocks::Vector{Blk})
    prev = workblk[x]
    id = getid(prev) + 1
    b = Blk(id, x, prev)
    push!(blocks, b)
    workblk[x] = b
    for q = queue
        sendblk!(q, b)
    end
    b
end

event_genBlk! (generic function with 1 method)

In [210]:
function event_arrivalBlk!(q::Queue, i::Int, workblk::Dict{Minor,AbstractBlk})
    x = getreceiver(q)
    b = receiveblk!(q, i)
    prev = workblk[x]
    if getid(prev) < getid(b)
        workblk[x] = b
    end
    b
end

event_arrivalBlk! (generic function with 1 method)

In [211]:
function rates(u1, u2, queue, lambda, mu)
#    u1 = sort(collect(keys(lambda)), by=x->x.label)
    r1 = [lambda[x] for x = u1]
#    u2 = sort(collect(keys(mu)), by=x->(x[1].label, x[2].label))
    r2 = [mu[x] * getlength(queue[x]) for x = u2]
    return r1, r2
end

rates (generic function with 1 method)

In [212]:
a = Minor("A")
b = Minor("B")

Minor("B")

In [424]:
lambda = Dict(
    a => 1/5,
    b => 1/10,
)

mu = Dict(
    (a,b) => 1/1,
    (b,a) => 1/1
)

Dict{Tuple{Minor,Minor},Float64} with 2 entries:
  (Minor("B"), Minor("A")) => 1.0
  (Minor("A"), Minor("B")) => 1.0

In [425]:
begin
    blocks = Vector{Blk}()

    events = 0
    u1 = sort(collect(keys(lambda)), by=x->x.label)
    u2 = sort(collect(keys(mu)), by=x->(x[1].label, x[2].label))
    workblk = Dict{Minor,AbstractBlk}(a=>Genesis(), b=>Genesis())
    qAtoB = Queue(a, b)
    qBtoA = Queue(b, a)
    queue = Dict(
        (a,b)=>qAtoB,
        (b,a)=>qBtoA,
    )
    input = Dict(
        a=>[qAtoB],
        b=>[qBtoA],
    )

    rng = MersenneTwister(9999)
    ctime = 0.0
    while ctime < 60*24*30 # 24 hours
        r1, r2 = rates(u1, u2, queue, lambda, mu)
        etime1 = [(r != 0) ? rand(rng, Exponential(1/r)) : Inf for r = r1]
        etime2 = [(r != 0) ? rand(rng, Exponential(1/r)) : Inf for r = r2]
        m1, i1 = findmin(etime1)
        m2, i2 = findmin(etime2)
        if m1 < m2
            ctime += m1
            event_genBlk!(u1[i1], workblk, input[u1[i1]], blocks)
        else
            ctime += m2
            v = rand(rng, DiscreteUniform(1,getlength(queue[u2[i2]])))
            event_arrivalBlk!(queue[u2[i2]], v, workblk)
        end
        events += 1
    end
end

In [426]:
@time workblk

  0.000000 seconds (3 allocations: 144 bytes)


Dict{Minor,AbstractBlk} with 2 entries:
  Minor("A") => (11818, "A")
  Minor("B") => (11818, "A")

In [427]:
workblk[b].prev

(11817, "A")

In [428]:
workblk[a].prev

(11817, "A")

In [429]:
blk = workblk[a]
chains = []
push!(chains, blk)
while blk != Genesis()
    blk = blk.prev
    push!(chains, blk)
end

In [430]:
chains

11819-element Array{Any,1}:
 (11818, "A")
 (11817, "A")
 (11816, "A")
 (11815, "A")
 (11814, "A")
 (11813, "A")
 (11812, "A")
 (11811, "B")
 (11810, "B")
 (11809, "B")
 (11808, "B")
 (11807, "A")
 (11806, "A")
 ⋮           
 (11, "A")   
 (10, "A")   
 (9, "A")    
 (8, "B")    
 (7, "B")    
 (6, "B")    
 (5, "A")    
 (4, "B")    
 (3, "A")    
 (2, "A")    
 (1, "A")    
 0           

In [431]:
blocks

13038-element Array{Blk,1}:
 (1, "A")    
 (2, "A")    
 (3, "A")    
 (4, "B")    
 (5, "A")    
 (6, "B")    
 (6, "A")    
 (7, "B")    
 (8, "B")    
 (9, "A")    
 (10, "A")   
 (11, "A")   
 (12, "A")   
 ⋮           
 (11809, "A")
 (11809, "B")
 (11810, "B")
 (11811, "B")
 (11812, "B")
 (11812, "A")
 (11813, "A")
 (11814, "A")
 (11815, "A")
 (11816, "A")
 (11817, "A")
 (11818, "A")

In [432]:
invchains = reverse(chains)

11819-element Array{Any,1}:
 0           
 (1, "A")    
 (2, "A")    
 (3, "A")    
 (4, "B")    
 (5, "A")    
 (6, "B")    
 (7, "B")    
 (8, "B")    
 (9, "A")    
 (10, "A")   
 (11, "A")   
 (12, "A")   
 ⋮           
 (11807, "A")
 (11808, "B")
 (11809, "B")
 (11810, "B")
 (11811, "B")
 (11812, "A")
 (11813, "A")
 (11814, "A")
 (11815, "A")
 (11816, "A")
 (11817, "A")
 (11818, "A")

In [433]:
open("chains.txt","w") do out #ファイルに書く
    print(out,invchains)
end

In [434]:
open("blocks.txt","w") do out #ファイルに書く
    print(out,blocks)
end

In [435]:
haki = 0
for i in 2:length(invchains)
    if invchains[i].minor == Minor("B")
        for j in 1:length(blocks)
            if (blocks[j].minor == Minor("A")) && (blocks[j].index == invchains[i].index)
                haki += 1
            end
        end
    end
end
haki

366

In [436]:
print("chains:",length(chains),"   blocks:",length(blocks),"   haki:",haki)

chains:11819   blocks:13038   haki:366